Adınız: Ceyda

Soyadınız: Metin

Okul Numarası: 2212721025

GitHub Repo Bağlantısı: https://github.com/ceydametinn/NBA_Flask

 Çoklu Doğrusal Regresyon ile NBA Oyuncu Puan Tahmini



---

## 1. Projenin Amacı
Bu projenin amacı, 2023-2024 sezonu NBA oyuncularının saha içi performans verilerini (oynanan süre, şut denemesi, yaş vb.) analiz ederek, bir oyuncunun maç başına atacağı ortalama sayıyı (**PTS**) tahmin eden bir Yapay Zeka modeli geliştirmektir.

Proje kapsamında veriler temizlenecek, kategorik veriler sayısal hale getirilecek ve **Backward Elimination (Geriye Doğru Eleme)** yöntemi ile modelin başarısını düşüren gereksiz değişkenler tespit edilip veri setinden çıkarılacaktır.

## 2. Kullanılan Veri Seti
Veri seti, NBA 2023-2024 normal sezon istatistiklerini içermektedir.
* **Hedef Değişken (Bağımlı):** `PTS` (Points Per Game)
* **Girdi Değişkenleri (Bağımsız):** `Age`, `MP` (Dakika), `FGA` (Şut Denemesi), `AST` (Asist) vb.

In [34]:
# Gerekli Kütüphanelerin Yüklenmesi
import pandas as pd  # Veri işleme
import numpy as np   # Matematiksel işlemler
import matplotlib.pyplot as plt # Görselleştirme
import seaborn as sns # İleri düzey görselleştirme
import statsmodels.api as sm # İstatistiksel modeller ve P-Value analizi için

# Makine Öğrenmesi için Scikit-Learn kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("Kütüphaneler başarıyla yüklendi.")

Kütüphaneler başarıyla yüklendi.


## 3. Veri Ön İşleme (Data Preprocessing)

Veri seti modele sokulmadan önce aşağıdaki işlemler uygulanmıştır:

1.  **Veri Sızıntısı (Data Leakage) Önlemi:** Hedef değişkenimiz olan `PTS` (Puan); matematiksel olarak `FG` (Basket), `3P` (Üçlük) ve `FT` (Serbest Atış) sütunlarının toplamından oluşur. Bu sütunları modele verirsek model ezber yapar. Bu nedenle sonucu doğrudan içeren bu sütunlar silinmiştir. Sadece "Deneme Sayıları" (`FGA`, `3PA` vb.) bırakılmıştır.
2.  **Gereksiz Veriler:** Oyuncu ismi (`Player`), Takım (`Tm`) gibi istatistiksel etkisi olmayan metin verileri çıkarılmıştır.
3.  **Kayıp Veri Analizi:** Eksik değerler (NaN), sütun ortalamaları ile doldurulmuştur.

In [35]:
# 1. Veri Setini Oku
filename = '2023-2024 NBA Player Stats - Regular.csv'

# Dosya okuma (Encoding hatası önlemi ile)
try:
    df = pd.read_csv(filename, encoding='latin-1', delimiter=';')
    if df.shape[1] < 5:
        df = pd.read_csv(filename, encoding='latin-1', delimiter=',')
except:
    df = pd.read_csv(filename, encoding='latin-1')

# 2. Temizlenecek Sütunlar (Drop List)
drop_list = [
    'Rk', 'Player', 'Tm', 'G', 'GS',  # Tanımlayıcı bilgiler (İsim, Takım vb.)
    'FG', '3P', '2P', 'FT',           # HİLELİ SÜTUNLAR (Cevabın kendisi)
    'FG%', '3P%', '2P%', 'eFG%', 'FT%', # Yüzdeler (Deneme sayısı daha değerlidir)
    'ORB', 'DRB', 'STL', 'BLK', 'TOV', 'PF' # Savunma istatistiklerini sadeleştirmek için çıkarıyoruz
]

# Sadece veri setinde mevcut olanları silelim
existing_drops = [col for col in drop_list if col in df.columns]
df_clean = df.drop(existing_drops, axis=1)

# 3. Eksik Verileri (NaN) Ortalama ile Doldur
df_clean = df_clean.fillna(df_clean.mean(numeric_only=True))

print("Veri Temizlendi. Kalan Sütunlar:")
print(df_clean.columns.tolist())

Veri Temizlendi. Kalan Sütunlar:
['Pos', 'Age', 'MP', 'FGA', '3PA', '2PA', 'FTA', 'TRB', 'AST', 'PTS']


### 3.1. Kategorik Değişkenlerin Kodlanması (Encoding)

Veri setindeki `Pos` (Pozisyon) sütunu (PG, C, SG gibi) metin formatındadır. Regresyon modeli matematiksel işlem yaptığı için bu veriler **One-Hot Encoding** yöntemiyle 0 ve 1'lerden oluşan sayısal sütunlara çevrilmiştir.

* **Drop First = True:** "Dummy Variable Trap" (Kukla Değişken Tuzağı) oluşmaması için ilk kategori sütunu düşürülmüştür.

In [36]:
# One-Hot Encoding Uygulama
df_encoded = pd.get_dummies(df_clean, columns=['Pos'], drop_first=True)

# Sonuç kontrolü
display(df_encoded.head())

,Age,MP,FGA,3PA,2PA,FTA,TRB,AST,PTS,Pos_C-PF,Pos_PF,Pos_PF-C,Pos_PF-SF,Pos_PG,Pos_PG-SG,Pos_SF,Pos_SF-PF,Pos_SF-SG,Pos_SG,Pos_SG-PG
0,24,21.9,6.3,1.3,5.0,1.5,6.6,1.3,7.6,False,False,True,False,False,False,False,False,False,False,False
1,24,17.5,6.8,1.9,4.9,1.7,5.4,1.8,7.7,False,False,False,False,False,False,False,False,False,False,False
2,24,24.2,6.1,1.0,5.1,1.4,7.2,1.1,7.6,False,True,False,False,False,False,False,False,False,False,False
3,26,34.0,14.3,0.6,13.7,5.5,10.4,3.9,19.3,False,False,False,False,False,False,False,False,False,False,False
4,23,21.0,5.6,2.7,2.8,0.7,2.8,1.1,5.8,False,False,False,False,False,False,False,False,False,True,False


In [37]:
# ---  BACKWARD ELIMINATION KODU ---
import statsmodels.api as sm
import numpy as np

# 1. HATA ÇÖZÜCÜ: Tüm verileri zorla sayıya (float) çevir
# Bu satır True/False değerlerini 1.0 ve 0.0 yapar
df_encoded = df_encoded.astype(float)

# 2. Veride sorun çıkaran sonsuz veya boş değer varsa temizle
df_encoded = df_encoded.replace([np.inf, -np.inf], np.nan).dropna()

print("Veri tipleri düzeltildi. Model kuruluyor...")

# 3. Hedef ve Girdileri Ayır
y = df_encoded['PTS']
X = df_encoded.drop('PTS', axis=1)

# 4. Sabit Katsayı Ekle
X_sm = sm.add_constant(X)

# 5. Modeli Kur ve Raporu Göster
model_ols = sm.OLS(y, X_sm).fit()
print(model_ols.summary())

Veri tipleri düzeltildi. Model kuruluyor...
                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     2461.
Date:                Fri, 12 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:55:48   Log-Likelihood:                -874.22
No. Observations:                 735   AIC:                             1788.
Df Residuals:                     715   BIC:                             1880.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [38]:
# --- ADIM 3: Backward Elimination (Gereksizleri Eleme) ---
import statsmodels.api as sm

# Tablodan tespit ettiğimiz P-değeri 0.05'ten büyük olanları listeye ekledik
cols_to_remove = [
    'FGA',       # 2PA ve 3PA ile çakıştığı için (P=0.758)
    'Pos_C-PF',  # P=0.863
    'Pos_PF-SF', # P=0.589
    'Pos_PF-C',  # P=0.357
    'AST',       # P=0.258
    'MP',        # P=0.114
    'Pos_PG-SG', # P=0.182
    'Pos_PF'     # P=0.087 (Sınırdan yüksek)
]

# Sütunları veri setinden (X) çıkarıyoruz
# errors='ignore' dedik ki eğer listedeki bir isim yoksa hata vermesin
X_optimized = X.drop(cols_to_remove, axis=1, errors='ignore')

# Modeli tekrar kuruyoruz (2. Tur)
X_sm_opt = sm.add_constant(X_optimized)
model_opt = sm.OLS(y, X_sm_opt).fit()

print(model_opt.summary())

                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     4244.
Date:                Fri, 12 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:55:48   Log-Likelihood:                -878.86
No. Observations:                 735   AIC:                             1782.
Df Residuals:                     723   BIC:                             1837.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8719      0.201     -4.328      0.0

In [39]:
# --- ADIM 3: Backward Elimination (Büyük Temizlik) ---
import statsmodels.api as sm

# Tablodan gördüğümüz P-değeri 0.05'ten büyük olanları (Gereksizleri) listeye ekledik
cols_to_remove = [
    'FGA',       # 2PA ve 3PA ile çakıştığı için (P=0.758)
    'Pos_C-PF',  # P=0.863 (Çok Yüksek)
    'Pos_SF-PF', # P=0.933 (Çok Yüksek)
    'Pos_PF-SF', # P=0.589
    'Pos_PF-C',  # P=0.357
    'Pos_SF-SG', # P=0.643
    'Pos_SG-PG', # P=0.633
    'Pos_PG-SG', # P=0.182
    'AST',       # P=0.258
    'MP'         # P=0.114
]

# Sütunları veri setinden (X) güvenli bir şekilde çıkarıyoruz
X_optimized = X.drop(cols_to_remove, axis=1, errors='ignore')

# Final Modeli Kuruyoruz
X_sm_opt = sm.add_constant(X_optimized)
model_opt = sm.OLS(y, X_sm_opt).fit()

print(model_opt.summary())

                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     5209.
Date:                Fri, 12 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:55:49   Log-Likelihood:                -878.31
No. Observations:                 735   AIC:                             1777.
Df Residuals:                     725   BIC:                             1823.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7880      0.213     -3.698      0.0

In [40]:
# --- ADIM 4: FİNAL TEMİZLİK VE MODEL KAYDI ---
import pickle
from sklearn.linear_model import LinearRegression

# 1. Son kalan yüksek P-değerini (Pos_PF: 0.215) temizliyoruz
X_final = X_optimized.drop(['Pos_PF'], axis=1, errors='ignore')

# 2. Final Modelin Eğitilmesi (Scikit-Learn ile)
# Web sitesinde kullanmak için Statsmodels yerine Sklearn kullanmak daha pratiktir.
regressor = LinearRegression()
regressor.fit(X_final, y)

print("🎉 TEBRİKLER! Model Başarıyla Eğitildi.")
print("Modelde Kullanılan Final Özellikler (Bunları not et):")
print(X_final.columns.tolist())

# 3. Modeli Dosyaya Kaydetme (.pkl)
# Bu 'model.pkl' dosyası web sitesinin beyni
pickle.dump(regressor, open('model.pkl', 'wb'))

print("\n💾 'model.pkl' dosyası oluşturuldu. Sol taraftaki dosyalar kısmını yenile!")

🎉 TEBRİKLER! Model Başarıyla Eğitildi.
Modelde Kullanılan Final Özellikler (Bunları not et):
['Age', '3PA', '2PA', 'FTA', 'TRB', 'Pos_PG', 'Pos_SF', 'Pos_SG']

💾 'model.pkl' dosyası oluşturuldu. Sol taraftaki dosyalar kısmını yenile!
